In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [15]:
def parametrized_pendulum(m, M, g, l, u_func):
    def pendulum(t, x):
        d = (m + M) - m*np.cos(x[2])
        return np.array([
            x[1],
            (m*(g - l*x[3]**2)*np.sin(x[2]) + u_func(x))/d,
            x[3],
            ((g*(m+M)/l - m*x[3]**2)*np.sin(x[2]) + u_func(x)/l)/d
        ])
    return pendulum

def lin_parametrized_pendulum(mu, u_func):
    def lin_pendulum(t, x):
        return np.array([
            x[1],
            mu*x[2] + u_func(x),
            x[3],
            (1+mu)*x[2] + u_func(x)
        ])
    return lin_pendulum

def lin_system_param(A, B, u, th):
    def lin_sys(t, x):
        return np.dot(A, x) + B * u(th, x)
    return lin_sys


In [17]:
mu = 1
l = 10
g = 9.8
A = np.array([
    [0, 1, 0, 0],
    [0, 0, -mu, 0],
    [0, 0, 0, 1],
    [0, 0, mu/l + g/l, 0]
])

B = np.array([0, 1, 0, 1/l])

sys = lin_system_param(A, B, u_control, np.array([0, 0, 0, 0]))

sys(0, np.array([0, 1, 0, 0]))



array([1., 0., 0., 0.])